In [1]:
%cd /ibex/user/slimhy/PADS/code
import os
import pickle
import numpy as np
from util.misc import fps_subsample
from datasets.sampling import sample_surface_tpp
from util.mesh import CUDAMesh
from tqdm import tqdm
import multiprocessing as mp


PART_INSTANCES = "/ibex/project/c2273/3DCoMPaT/manifold_part_instances/"
OUT_DIR = "/ibex/project/c2273/PADS/3DCoMPaT/part_points"
POINTS_PER_PART = 512
FPS_RATIO = 0.25
N_PROCS = 12
N_NODES = 4
PROC_ID = 0


def sample_part_points(pkl_file):
    mesh_dict = pickle.load(open(os.path.join(PART_INSTANCES, pkl_file), "rb"))
    model_name = pkl_file.split(".")[0]
    
    points_dict = {}
    for mesh_k, mesh in mesh_dict.items():
        cuda_mesh = CUDAMesh.from_trimesh(mesh).to("cpu")
        p_points = sample_surface_tpp(cuda_mesh, POINTS_PER_PART)
        p_points = fps_subsample(p_points / FPS_RATIO, ratio=FPS_RATIO)
        points_dict[mesh_k] = p_points.cpu().squeeze()

    np.save(f"{OUT_DIR}/{model_name}.npy", points_dict)
    return pkl_file


/ibex/user/slimhy/PADS/code


No CUDA runtime is found, using CUDA_HOME='/sw/rl9g/cuda/12.2/rl9_binary'


In [2]:
pkl_files = [f for f in os.listdir(PART_INSTANCES) if f.endswith(".pkl")]
pkl_files.sort()


def chunk(l, n):
    """
    Chunk a list into n equally sized sublists.
    And NOT chunk it into sublists of size n.
    """
    return [l[i*len(l)//n:(i+1)*len(l)//n] for i in range(n)]

pkl_files = chunk(pkl_files, N_NODES)[PROC_ID]

# Create a pool of workers
with mp.Pool(processes=N_PROCS) as pool:
    # Use imap to process files and update the progress bar
    for _ in tqdm(pool.imap_unordered(sample_part_points, pkl_files), total=len(pkl_files), desc="Processing files"):
        pass

Processing files: 100%|██████████| 2502/2502 [1:29:10<00:00,  2.14s/it]  
